In [2]:
testfolder = "/Users/iwatano/Desktop/tmp_PRESUME/20201122_debug" # <- specify a filepath for testing PRESUME
PRESUME = "/Users/iwatano/Desktop/Dev/yachielab/PRESUME/PRESUME.py" # <- specify a filepath of PRESUME.py 

In [3]:
!python3 $PRESUME -h


usage: PRESUME.py [-h] [--param PARAM] [-V] [--monitor MONITOR] [-n N]
                  [--tree TREE] [-L L] [-f F] [--polyC] [-d D] [-s S] [-T T]
                  [-e E] [--gtrgamma GTRGAMMA] [-u U] [--ud UD] [--ld LD]
                  [-m M] [--constant CONSTANT] [--output OUTPUT] [--qsub]
                  [--idANC IDANC] [--tMorigin TMORIGIN] [--debug] [--bar]
                  [--viewANC] [--save] [--CV] [-r R] [--seed SEED]
                  [--inprob INPROB] [--inlength INLENGTH] [--delprob DELPROB]
                  [--dellength DELLENGTH] [--indels INDELS] [--dop DOP]
                  [--dist DIST] [--editprofile EDITPROFILE]

PRESUME.py

optional arguments:
  -h, --help            show this help message and exit
  --param PARAM         load argument file(csv file)
  -V, --version
  --monitor MONITOR     time limit (default=None)
  -n N                  required number of sequences: if you specified this

                        number of the sequence reach the specified 

In [5]:
out = "{}/test_1".format(testfolder)
!python3 $PRESUME --output $out -n 100 --gtrgamma default --save 

Substitution rate matrix:
[[-0.0249975  0.0083325  0.0083325  0.0083325]
 [ 0.0083325 -0.0249975  0.0083325  0.0083325]
 [ 0.0083325  0.0083325 -0.0249975  0.0083325]
 [ 0.0083325  0.0083325  0.0083325 -0.0249975]]

######     ######     #######     #####     #     #    #     #    #######
#     #    #     #    #          #     #    #     #    ##   ##    #
#     #    #     #    #          #          #     #    # # # #    #
######     ######     #####       #####     #     #    #  #  #    #####
#          #   #      #                #    #     #    #     #    #
#          #    #     #          #     #    #     #    #     #    #
#          #     #    #######     #####      #####     #     #    #######
Version:     1.0.0
Last update: April 24, 2020
GitHub:      https://github.com/yachielab/PRESUME

Generating a FASTA file...
Generating a Newick file......

Simulation end time point:         7.0
Number of generated sequences:     128
Seed for random number generation: 2431

Number of retria

In [ ]:
out = "{}/test_2".format(testfolder)
!python3 $PRESUME --seed 0 -L 1000 -n 1000 -e 0.01 --gtrgamma default --output $out

In [ ]:
out = "{}/test_3".format(testfolder)
!python3 $PRESUME --seed 0 -L 1000 -n 1000 -s 0.1 --CV --gtrgamma default --output $out

In [ ]:
out = "{}/test_4".format(testfolder)
inprob = "/Users/iwatano/Desktop/tmp_PRESUME/20201122_debug/test_4/prob.txt"
length = "/Users/iwatano/Desktop/tmp_PRESUME/20201122_debug/test_4/length.txt"
!python3 $PRESUME -n 100 --seed 0 --output $out --inprob $inprob --inlength $length  --delprob $inprob --dellength $length --gtrgamma default


In [ ]:
out = "{}/test_5".format(testfolder)
test_input = "/Users/iwatano/Desktop/tmp_PRESUME/20201122_debug/test_5/test_input.csv"
!python3 $PRESUME -n 100 --seed 0 --output $out --editprofile $test_input

In [ ]:
out = "{}/test_6".format(testfolder)
!python3 $PRESUME --seed 0 -L 1000 -n 1000 --constant 0.1 --output $out --qsub --debug

In [ ]:
out = "{}/tmp_06".format(testfolder)
!python3 $PRESUME --monitor 12.0 -L 1000 -d 1.0 -s 0 -T 1000 -e 0 -u 1048576 --idANC 32 --tMorigin 5.0 --seed 3972 --dist gamma2 --constant 0.1 --output $out


In [ ]:
import numpy as np
def sub_mat_parser(infile):

        # substitution matrix
        # = [
        # A: [AtoA, AtoT, AtoG, AtoC]
        # C: [CtoA, CtoT, CtoG, CtoC]
        # G: [GtoA, GtoT, GtoG, GtoC]
        # T: [TtoA, TtoT, TtoG, TtoC]
        # ] * L

        base = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

        with open(infile, 'r') as handle:
            
            row = 0
            for line in handle:

                if (row == 0):
                    
                    substitution_prob_matrix = np.zeros((1000, 4, 4))
                
                else:

                    transition = line.split(",")[0] # eg. 'CT'
                    prob_list  = [float(prob_str) for prob_str in line.split(",")[1:]]

                    for pos, prob in enumerate(prob_list):

                        if (pos < 1000):
                            substitution_prob_matrix[pos][base[transition[0]]][base[transition[1]]] = prob
                        else:
                            print("args_reader.py: Edit profile assumes longer sequence than the actual sequence length")
                            sys.exit(1)

                row       += 1
        
        for pos in range(1000):
            
            for i in range(4):

                change_prob = 0
                for j in range(4):
                    if (i != j):
                        change_prob += substitution_prob_matrix[pos, i, j] 

                if (change_prob > 1):
                    print("args_reader.py: Editing probability > 1!")
                    sys.exit(1)

                substitution_prob_matrix[pos, i, i] = 1 - change_prob
        
        #for matrix in substitution_prob_matrix:
        #    print (matrix)
        return substitution_prob_matrix

submat = sub_mat_parser("/Users/iwatano/Desktop/Dev/yachielab/PRESUME/example/example_5/transition_probability")